<a href="https://colab.research.google.com/github/tienhuynh96/NMT_Neural-Machine-Translation/blob/main/%5Bdemo%5D_NMT_Seq2Seq_RNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Machine Translation using RNNs**

##**Dataset**

In [ ]:
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.1 MB/s eta 0:00:00


##**1.1 Load dataset from dataset library**

The IWSLT'15 English-Vietnamese data is used from Stanford NLP group.

In [ ]:
# Load dataset
# The IWSLT'15 English-Vietnamese data is used from Stanford NLP group.
from datasets import load_dataset

data = load_dataset(
    "mt_eng_vietnamese",
    "iwslt2015-en-vi"
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for mt_eng_vietnamese contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mt_eng_vietnamese
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warn

ConnectionError: Couldn't reach https://nlp.stanford.edu/projects/nmt/data/iwslt15.en-vi/train.en (error 403)

In [ ]:
data['train']

In [ ]:
data['train']['translation'][0]

##**1.2 Incase can not load dataset from dataset**

Wget data from github: https://github.com/stefan-it/nmt-en-vi

In [ ]:
# Incase can not load dataset from dataset
# Try
# Wget data from github: https://github.com/stefan-it/nmt-en-vi
# Train file
!wget "https://github.com/stefan-it/nmt-en-vi/raw/master/data/train-en-vi.tgz"
# Dev file
!wget "https://github.com/stefan-it/nmt-en-vi/raw/master/data/dev-2012-en-vi.tgz"
# Test file
!wget "https://github.com/stefan-it/nmt-en-vi/raw/master/data/test-2013-en-vi.tgz"


--2024-05-29 02:28:55--  https://github.com/stefan-it/nmt-en-vi/raw/master/data/train-en-vi.tgz
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/stefan-it/nmt-en-vi/master/data/train-en-vi.tgz [following]
--2024-05-29 02:28:56--  https://raw.githubusercontent.com/stefan-it/nmt-en-vi/master/data/train-en-vi.tgz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9903559 (9.4M) [application/octet-stream]
Saving to: ‘train-en-vi.tgz’

train-en-vi.tgz     100%[===================>]   9.44M  --.-KB/s    in 0.07s   

2024-05-29 02:28:58 (130 MB/s) - ‘train-en-vi.tgz’ saved [9903559/9903559]

--2024-05-29 0

In [ ]:
# extract the tgz file
!tar -xzf train-en-vi.tgz
!tar -xzf dev-2012-en-vi.tgz
!tar -xzf test-2013-en-vi.tgz

In [ ]:
# Create a dataset shape similar with datashape which download from datasets library
data = {
    "train": {
                "translation":[]
              },
    "validation": {
                "translation":[]
              },
    "test": {
                "translation":[]
              },
}

In [ ]:
data

{'train': {'translation': []},
 'validation': {'translation': []},
 'test': {'translation': []}}

In [ ]:
# Add path for train, test, validation file
# Train file
train_source_file = "/content/train.en"
train_target_file = "/content/train.vi"
# Validation file
val_source_file = "/content/tst2012.en"
val_target_file = "/content/tst2012.vi"
# Test file
test_source_file = "/content/tst2013.en"
test_target_file = "/content/tst2013.vi"

In [ ]:
# Build function: Get examples from sourse, target file and append into data
def generate_examples(source_file, target_file, data, dataname):
    # Open and read source file and target file of "train, test, validation" data
    with open(source_file, encoding="utf-8") as f:
        source_sentences = f.read().split("\n")
    with open(target_file, encoding="utf-8") as f:
        target_sentences = f.read().split("\n")

    # Add examples of "train, test, validation" data
    data_gen = data[dataname]['translation']
    source, target = "en", "vi"
    for idx, (l1, l2) in enumerate(zip(source_sentences, target_sentences)):
        result = {source: l1, target: l2}
        data_gen.append(result)
    return data

In [ ]:
data = generate_examples(source_file=train_source_file, target_file=train_target_file,
                        data=data, dataname="train")

data = generate_examples(test_source_file, test_target_file, data, "test")
data = generate_examples(val_source_file, val_target_file, data, "validation")


In [ ]:
data['train']['translation'][0]

{'en': 'Rachel Pike : The science behind a climate headline',
 'vi': 'Khoa học đằng sau một tiêu đề về khí hậu'}

In [ ]:
data['test']['translation'][0]


{'en': 'When I was little , I thought my country was the best on the planet , and I grew up singing a song called &quot; Nothing To Envy . &quot;',
 'vi': 'Khi tôi còn nhỏ , Tôi nghĩ rằng BắcTriều Tiên là đất nước tốt nhất trên thế giới và tôi thường hát bài &quot; Chúng ta chẳng có gì phải ghen tị . &quot;'}

In [ ]:
data['validation']['translation'][0]

{'en': 'How can I speak in 10 minutes about the bonds of women over three generations , about how the astonishing strength of those bonds took hold in the life of a four-year-old girl huddled with her young sister , her mother and her grandmother for five days and nights in a small boat in the China Sea more than 30 years ago , bonds that took hold in the life of that small girl and never let go -- that small girl now living in San Francisco and speaking to you today ?',
 'vi': 'Làm sao tôi có thể trình bày trong 10 phút về sợi dây liên kết những người phụ nữ qua ba thế hệ , về việc làm thế nào những sợi dây mạnh mẽ đáng kinh ngạc ấy đã níu chặt lấy cuộc sống của một cô bé bốn tuổi co quắp với đứa em gái nhỏ của cô bé , với mẹ và bà trong suốt năm ngày đêm trên con thuyền nhỏ lênh đênh trên Biển Đông hơn 30 năm trước , những sợi dây liên kết đã níu lấy cuộc đời cô bé ấy và không bao giờ rời đi -- cô bé ấy giờ sống ở San Francisco và đang nói chuyện với các bạn hôm nay ?'}

##**Tokenization**

In [ ]:
# Get_tokenizer and build vocab
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

/usr/local/lib/python3.10/dist-packages/torchtext/data/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/usr/local/lib/python3.10/dist-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/usr/local/lib/python3.10/dist-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated

In [ ]:
# Set source and target language
SRC_LANGUAGE = 'en'
TGT_LANGUAGE = 'vi'

# Create token_transform and vocab_transform is empty dictionary
token_transform = {}
vocab_transform = {}

# Add tokenizer for source and target language
token_transform[SRC_LANGUAGE] = get_tokenizer('basic_english')
token_transform[TGT_LANGUAGE] = get_tokenizer('basic_english')

# Set special symbols and its id
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
special_symbols = ['<unk>', '<pad>', '<bos>','eos']


In [ ]:
token_transform

{'en': <function torchtext.data.utils._basic_english_normalize(line)>,
 'vi': <function torchtext.data.utils._basic_english_normalize(line)>}

In [ ]:
# Build vocab
# Build yield tokens function: input data_iter and language (en or vi)
def yield_tokens(data_iter, lang):
  for data_sample in data_iter['translation']:
    yield token_transform[lang](data_sample[lang])

# Build vocab for source and target language
for lang in [SRC_LANGUAGE, TGT_LANGUAGE]:
  # Get data from train data
  train_iter = data['train']

  # Create torchtext's vocab object
  vocab_transform[lang] = build_vocab_from_iterator(
      yield_tokens(train_iter, lang),   # yield for each language
      min_freq=1,               # Minimum of appear tokens in vocab is stored
      specials=special_symbols, # Get special symbols
      special_first=True        # Set special symbol ids is first
  )

  # Set set_default_index is UNK_IDX
  vocab_transform[lang].set_default_index(UNK_IDX)

In [ ]:
# Build vocab for source language
vocab_transform[SRC_LANGUAGE].get_itos()[:10]

['<unk>', '<pad>', '<bos>', 'eos', ',', '.', 'the', 'and', 'to', '&apos']

In [ ]:
# Build vocab for target language
vocab_transform[TGT_LANGUAGE].get_itos()[:10]


['<unk>', '<pad>', '<bos>', 'eos', ',', '.', 'và', 'tôi', 'là', 'một']

In [ ]:
len(vocab_transform[SRC_LANGUAGE]), len(vocab_transform[TGT_LANGUAGE])

(47270, 21113)

##**Dataloader**

In [ ]:
import torch
from torch.nn.utils.rnn import pad_sequence     # padding follow the highest sequence length in dataset

# Set max len
MAX_LEN = 100

# helper function to club together sequential operations
# Take a series of transformation functions as arguments and returns a new function
def sequential_transforms(*transforms):   # *transforms allows the function to accept any number of transformation functions as arguments.
    def func(txt_input): # inner function
        # Applying Transformations in Sequence
        # For each transformation, it applies the function to txt_input, updating txt_input with the transformed result.
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func


# function to add BOS/EOS and create tensor for input sequence indices: BOS-ids-EOS
def tensor_transform(token_ids):
    return torch.cat((torch.tensor([BOS_IDX]),
                      torch.tensor(token_ids),
                      torch.tensor([EOS_IDX])))

# Function to truncate
def truncate(sample):
    if sample.size(0) > MAX_LEN:
        return sample[:MAX_LEN, :]
    else:
        return sample



In [ ]:
# ``src`` and ``tgt`` language text transforms to convert raw strings into tensors indices
text_transform = {}
for lang in [SRC_LANGUAGE, TGT_LANGUAGE]:
    text_transform[lang] = sequential_transforms(
        token_transform[lang], # Tokenization   => tokens
        vocab_transform[lang], # Numericalization => ids
        tensor_transform # Add BOS/EOS and create tensor  => bos ids eos
    )

# function to collate data samples into batch tensors
def collate_fn(batch):
    src_batch, tgt_batch = [], []
    # transforms for each sample in batch
    for sample in batch:
        src_sample, tgt_sample = sample[SRC_LANGUAGE], sample[TGT_LANGUAGE]
        src_batch.append(text_transform[SRC_LANGUAGE](src_sample).to(dtype=torch.int64))
        tgt_batch.append(text_transform[TGT_LANGUAGE](tgt_sample).to(dtype=torch.int64))

    # Add pad sequence and truncate
    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX, batch_first=True)
    src_batch = truncate(src_batch)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX, batch_first=True)
    tgt_batch = truncate(tgt_batch)
    return src_batch, tgt_batch

In [ ]:
from torch.utils.data import DataLoader

# Set batch size
BATCH_SIZE = 32

# Build dataloader for train, validation, test data
train_dataloader = DataLoader(
    data['train']['translation'],
    batch_size=BATCH_SIZE,
    collate_fn=collate_fn
)

valid_dataloader = DataLoader(
    data['validation']['translation'],
    batch_size=BATCH_SIZE,
    collate_fn=collate_fn
)

test_dataloader = DataLoader(
    data['test']['translation'],
    batch_size=BATCH_SIZE,
    collate_fn=collate_fn
)

In [ ]:
# Check the data of first batch
src_ids, tgt_ids = next(iter(train_dataloader))

In [ ]:
# Check shape
src_ids.shape, tgt_ids.shape

(torch.Size([32, 52]), torch.Size([32, 78]))

In [ ]:
src_ids[0]

tensor([    2,  6429, 17576,     6,   295,   553,    11,   682,  5334,     3,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1])

In [ ]:
# Decode the source ids
' '.join([vocab_transform[SRC_LANGUAGE].lookup_token(token) for token in src_ids[0]])

'<bos> rachel pike the science behind a climate headline eos <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>'

In [ ]:
# Decode the target ids
' '.join([vocab_transform[TGT_LANGUAGE].lookup_token(token) for token in tgt_ids[0]])

'<bos> khoa học đằng sau một tiêu đề về khí hậu eos <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>'

##**Model**

##**4.1 Build encoder**

In [ ]:
# Build encoder fucntion
import torch.nn as nn

class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_p=0.1):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size        # hidden size of RNN

        self.embedding = nn.Embedding(input_size, hidden_size)        # Covert to embedding
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True) # Use GRU
        self.dropout = nn.Dropout(dropout_p)  # Dropout

    def forward(self, input):
        embedded = self.dropout(self.embedding(input))  # b x s x h
        output, hidden = self.gru(embedded)             # output: b x s x h and hidden:number layer x b x h
        return output, hidden

In [ ]:
# Set encoder
input_size = len(vocab_transform[SRC_LANGUAGE])
hidden_size = 256

encoder = EncoderRNN(input_size, hidden_size)

In [ ]:
# Check encoder function
encoder_output, encoder_hidden = encoder(src_ids)
encoder_output.shape, encoder_hidden.shape

(torch.Size([32, 52, 256]), torch.Size([1, 32, 256]))

##**4.2 Build Decoder**

In [ ]:
# Build Decoder fucntion
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)          # Output of decoder

    def forward(self, input, hidden):
        output = self.embedding(input)            # input: target sentence
        output, hidden = self.gru(output, hidden) # output: b x s x h and hidden:number layer x b x h
        output = self.out(output)                 # compute liner for each sequence (s) in output
        return output, hidden

In [ ]:
# Set Decoder
output_size = len(vocab_transform[TGT_LANGUAGE])
hidden_size = 256

decoder = DecoderRNN(hidden_size, output_size)

In [ ]:
# Check Decoder function
decoder_output, decoder_hidden = decoder(tgt_ids, encoder_hidden)
decoder_output.shape, decoder_hidden.shape

(torch.Size([32, 78, 21113]), torch.Size([1, 32, 256]))

In [ ]:
decoder_input = torch.empty(32, 1, dtype=torch.long).fill_(BOS_IDX)

In [ ]:
decoder_input

tensor([[2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2]])

##**4.3 Build sequence to sequence function**

In [ ]:
# Build sequence to sequence fucntion
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device, BOS_IDX):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        self.BOS_IDX = BOS_IDX

    def forward(self, src_ids, tgt_ids):
        # Get batch size
        batch_size = tgt_ids.size(0)
        # Get sequence length
        seq_len = tgt_ids.size(1)

        # Create decoder input (Batch size, 1), each input begin by BOS_IDS
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=self.device).fill_(self.BOS_IDX)
        # Compute encoder ouput and encoder hidden (encoder hidden = decoder hidden)
        encoder_output, decoder_hidden = self.encoder(src_ids)
        # Create empty list of  decoder output
        decoder_outputs = []

        # Iterator each token in target sentence
        for i in range(seq_len):
            # Compute decoder output and update decoder_hidden
            decoder_output, decoder_hidden  = self.decoder(decoder_input, decoder_hidden)     # output: b x s x h and hidden:number layer x b x h
            # Update decoder output
            decoder_outputs.append(decoder_output)

            # Teacher forcing: Feed the target as the next input
            decoder_input = tgt_ids[:, i].unsqueeze(1) # Teacher forcing, unsqueeze(1) to get shape (batct, 1)

        # Cat the decoder ouputs
        decoder_outputs = torch.cat(decoder_outputs, dim=1)   # concatenate at sequence dimention, decoder_outputs is list => use torch.cat to concatenate
        return decoder_outputs, decoder_hidden

In [ ]:
# Build model
input_size = len(vocab_transform[SRC_LANGUAGE])
output_size = len(vocab_transform[TGT_LANGUAGE])
hidden_size = 256
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Build encoder
encoder = EncoderRNN(input_size, hidden_size)
# Build decoder
decoder = DecoderRNN(hidden_size, output_size)
# Build model = sequence to sequence
model = Seq2Seq(encoder, decoder, device, EOS_IDX)
model.to(device)

Seq2Seq(
  (encoder): EncoderRNN(
    (embedding): Embedding(47270, 256)
    (gru): GRU(256, 256, batch_first=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (decoder): DecoderRNN(
    (embedding): Embedding(21113, 256)
    (gru): GRU(256, 256, batch_first=True)
    (out): Linear(in_features=256, out_features=21113, bias=True)
  )
)

In [ ]:
# Target input is sequence except the last token
tgt_input = tgt_ids[:, :-1]
# Target ouput is sequence except the first token (BOS)
tgt_output = tgt_ids[:, 1:]

# Compute decoder_outputs, decoder_hidden of model
decoder_outputs, decoder_hidden = model(src_ids.to(device), tgt_input.to(device))

In [ ]:
# Check shape
decoder_outputs.shape, decoder_hidden.shape

(torch.Size([32, 77, 21113]), torch.Size([1, 32, 256]))

In [ ]:
tgt_output.shape

torch.Size([32, 77])

In [ ]:
# Define loss = Cross Entropy Loss
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

In [ ]:
# Reshape decoder_outputs = (Batch_size * Sequence length, hidden)
decoder_outputs.reshape(-1, decoder_outputs.shape[-1]).shape

torch.Size([2464, 21113])

In [ ]:
# Reshape tgt_output = (Batch_size * Sequence length, 1)
tgt_output.reshape(-1).shape

torch.Size([2464])

In [ ]:
# Compute loss
loss = criterion(decoder_outputs.reshape(-1, decoder_outputs.shape[-1]), tgt_output.reshape(-1).to(device))
loss

tensor(9.9645, device='cuda:0', grad_fn=<NllLossBackward0>)

##**Trainer**

In [ ]:
import time

# Creaete train epoch function
def train_epoch(model, optimizer, criterion, train_dataloader, device):
    model.train()
    losses = []

    # Iterator to get data from train_loader
    for idx, (src_ids, tgt_ids) in enumerate(train_dataloader):
        # put src_ids, tgt_ids to device
        src_ids = src_ids.to(device)
        tgt_ids = tgt_ids.to(device)

        # Get input and output target
        tgt_input = tgt_ids[:, :-1]
        tgt_output = tgt_ids[:, 1:]

        # Set optimizer
        optimizer.zero_grad()
        # Compute decoder_outputs, decoder_hidden
        decoder_outputs, decoder_hidden = model(src_ids, tgt_input)

        # Compute loss
        loss = criterion(
            decoder_outputs.reshape(-1, decoder_outputs.shape[-1]),
            tgt_output.reshape(-1))
        # backward
        loss.backward()

        # weight update
        optimizer.step()
        # Append loss
        losses.append(loss.item())

    return sum(losses) / len(losses)

# Creaete evaluate epoch function
def evaluate(model, data_loader, criterion, device):
    model.eval()
    losses = []
    with torch.no_grad():
        for idx, (src_ids, tgt_ids) in enumerate(data_loader):
            # put src_ids, tgt_ids to device
            src_ids = src_ids.to(device)
            tgt_ids = tgt_ids.to(device)
            # Get input and output target
            tgt_input = tgt_ids[:, :-1]
            tgt_output = tgt_ids[:, 1:]

            # Compute decoder_outputs, decoder_hidden
            decoder_outputs, decoder_hidden = model(src_ids, tgt_input)
            loss = criterion(
                decoder_outputs.reshape(-1, decoder_outputs.shape[-1]),
                tgt_output.reshape(-1)
            )
            # Append loss
            losses.append(loss.item())
    return sum(losses) / len(losses)

# Creaete train function
def train(model, train_dataloader, valid_dataloader, optimizer, criterion, device, epochs):
    # Iterator for each epoch
    for epoch in range(1, epochs+1):
        # Set start time
        start_time = time.time()
        # Compute train loss
        train_loss = train_epoch(model, optimizer, criterion, train_dataloader, device)
        # Compute validatioin loss
        valid_loss = evaluate(model, valid_dataloader, criterion, device)
        # Set end time
        end_time = time.time()
        print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {valid_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))

##**Training**

In [ ]:
import torch.optim as optim

# Create parameter input
input_size = len(vocab_transform[SRC_LANGUAGE])
output_size = len(vocab_transform[TGT_LANGUAGE])
hidden_size = 256
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create encoder and decoder
encoder = EncoderRNN(input_size, hidden_size)
decoder = DecoderRNN(hidden_size, output_size)
# Create model
model = Seq2Seq(encoder, decoder, device, EOS_IDX)
model.to(device)

# Set optimizer, loss and epochs
epochs = 10
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)
optimizer = optim.Adam(model.parameters())
# Train model
train(model, train_dataloader, valid_dataloader, optimizer, criterion, device, epochs)

Epoch: 1, Train loss: 5.521, Val loss: 5.186, Epoch time = 550.014s
Epoch: 2, Train loss: 5.001, Val loss: 5.023, Epoch time = 547.630s
Epoch: 3, Train loss: 4.779, Val loss: 4.960, Epoch time = 549.021s
Epoch: 4, Train loss: 4.630, Val loss: 4.935, Epoch time = 547.652s
Epoch: 5, Train loss: 4.518, Val loss: 4.932, Epoch time = 546.922s
Epoch: 6, Train loss: 4.428, Val loss: 4.910, Epoch time = 545.556s
Epoch: 7, Train loss: 4.354, Val loss: 4.918, Epoch time = 547.581s
Epoch: 8, Train loss: 4.291, Val loss: 4.935, Epoch time = 547.704s
Epoch: 9, Train loss: 4.238, Val loss: 4.941, Epoch time = 550.145s
Epoch: 10, Train loss: 4.191, Val loss: 4.953, Epoch time = 549.088s
